## Notebook 4 - Location Analysis

In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [4]:
import re

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

In [6]:
#pip install -U regex

In [5]:
pd.set_option("max_colwidth", 100)

In [6]:
from google.cloud import storage
# !pip install gcsfs --upgrade

In [8]:
pip install a-world-of-countries

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import awoc
my_world = awoc.AWOC()

### Setting Up Helper Functions

In [10]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [11]:
twitter_df5= spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/kshitijm/Final_Project-FiltData3')

In [12]:
twitter_df5.limit(15)

23/03/10 14:52:48 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


id,created_at,text,tweet_text,tweet_coordinates,tweet_country,tweet_country_code,tweet_place_full_name,tweet_place_type,is_reply,is_quote,is_retweeted,reply_count,quote_count,retweet_count,favorite_count,rp_original_id,rp_user_id,rp_original_user,qu_original_id,rt_original_id,rt_original_user,account_id,account_name,account_description,account_location,is_verified,total_followers,total_friends,total_listed,total_favourites,total_tweets,account_created_at,account_profile_picture,hashtags,tweet_type
1566378682318462976,Sun Sep 04 10:52:...,RT @C_A_Graham: H...,had another conve...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566034383559065601,C_A_Graham,483864332,erin_gilbey,Storyteller and 1...,"Birmingham, England",false,442,601,8,15433,8431,Sun Feb 05 13:37:...,https://pbs.twimg...,[],retweet
1566378688698097664,Sun Sep 04 10:52:...,RT @Oludeewon: I ...,i hope you have s...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566165377796771840,Oludeewon,984182548791164928,Odeshiye,♎ ...So A Child M...,Ghana,false,4902,4613,3,149571,92052,Wed Apr 11 21:32:...,https://pbs.twimg...,[],retweet
1566378715709427719,Sun Sep 04 10:52:...,RT @JackPosobiec:...,breaking: @trinit...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1565431877460189189,JackPosobiec,1558859689789644800,EuroAmerican87,American by birth...,Anywhere but here...,false,6,34,1,50,89,Sun Aug 14 16:54:...,https://pbs.twimg...,[],retweet
1566378716887916546,Sun Sep 04 10:52:...,RT @amishra77: Th...,the appalling con...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566378309520596993,amishra77,1279643153474224130,Bharath12450842,One of the 900 mi...,null,false,193,502,0,6244,9146,Sun Jul 05 05:09:...,https://pbs.twimg...,[],retweet
1566378724135555072,Sun Sep 04 10:52:...,RT @imzarvinnn: S...,senior high schoo...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566034947831660545,imzarvinnn,1530893925803622401,lvchdvmp,I'm counting on y...,archen's heart,false,341,458,0,11632,5624,Sun May 29 12:49:...,https://pbs.twimg...,[],retweet
1566378734529101824,Sun Sep 04 10:52:...,RT @imzarvinnn: S...,senior high schoo...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566034947831660545,imzarvinnn,1398282464854827012,doyiea_,she/her ...,null,false,1,91,0,59,621,Fri May 28 14:18:...,https://pbs.twimg...,[],retweet
1566378741508734976,Sun Sep 04 10:52:...,You can also tag ...,you can also tag ...,null,null,null,null,null,true,false,,0,0,0,0,1566378738438533120,1299666479344807937,thxteacher,null,null,null,1299666479344807937,thxteacher,Our Teachers are ...,null,false,902,69,1,325,1257,Sat Aug 29 11:13:...,https://pbs.twimg...,"[TeachersDay, Tha...",reply
1566378743710744576,Sun Sep 04 10:52:...,RT @cutedevil5934...,"""teachers play a ...",null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566366649912938498,cutedevil5934,769516988335067137,shoaibAlam28,http://Instagram....,"Jamui, India",false,85,146,0,1434,5245,Sat Aug 27 12:48:...,https://pbs.twimg...,[],retweet
1566378745539469312,Sun Sep 04 10:52:...,@jenqissante ! 🧚...,@jenqissante ! 🧚...,null,null,null,null,null,true,true,,0,0,0,0,1566373182810234882,1547185772708909056,jenqissante,1496397166339932162,null,null,1307586299893702656,PaperupPh,more than 1k+ wor...,10AM-10PM,false,520,182,2,95,820,Sun Sep 20 07:44:...,https://pbs.twimg...,[],quote
1566378750090027008,Sun Sep 04 10:52:...,RT @SethAbramson:...,like... one of hi...,null,null,null,null,null,false,false,RT,0,0,0,0,null,null,null,null,1566236400785264640,SethAbramson,1269070500082450437,ResisterNell,"Activist, #Resist...",Kentucky,false,9745,10379,16,463255,333552,Sat Jun 06 00:57:...,https://pbs.twimg...,[],retweet


In [11]:
twitter_df5.count()

27102994

In [13]:
twitter_df5.select([count(when(isnull(c), c)).alias(c) for c in twitter_df5.columns])

id,created_at,text,tweet_text,tweet_coordinates,tweet_country,tweet_country_code,tweet_place_full_name,tweet_place_type,is_reply,is_quote,is_retweeted,reply_count,quote_count,retweet_count,favorite_count,rp_original_id,rp_user_id,rp_original_user,qu_original_id,rt_original_id,rt_original_user,account_id,account_name,account_description,account_location,is_verified,total_followers,total_friends,total_listed,total_favourites,total_tweets,account_created_at,account_profile_picture,hashtags,tweet_type
0,0,0,0,27081980,26869044,26869044,26869044,26869044,0,0,0,0,0,0,0,22697695,22571090,22571090,24666475,9041884,9041884,0,0,0,8218657,0,0,0,0,0,0,0,0,0,0


In [17]:
# Out of 27 million data rows, we have account locations for 8 million rows only

In [14]:
twitter_df5.select('account_id','account_location').distinct().count()

7869779

In [16]:
twitter_df5.select('account_id').distinct().count()
# For some accounts, the location is changing with time

7418754

In [9]:
twitter_df5.select('account_id','account_location').distinct().limit(10)

account_id,account_location
4439290607,️️️️ ️️ ️️️️ ️
67313763,upstate New York
1474387239710990352,null
1141603477,"Arlington, Va"
118899225,"Milwaukee, WI"
1145097164902883331,he/him • 22 • l(g...
1477937988822872067,"London, England"
1320696007672451079,null
306975891,"Manhattan, KS"
1457000608423100431,null


In [43]:
# Selecting distinct users with their locations from our tweets data

df_loc = twitter_df5.select('account_id','account_location').distinct()\
.groupBy('account_location')\
.agg(count('*').alias('total_twitterers'))

df_loc.orderBy('total_twitterers',ascending=False).show()

+----------------+----------------+
|account_location|total_twitterers|
+----------------+----------------+
|            null|         2492747|
|   United States|           66437|
|  Lagos, Nigeria|           40269|
| Los Angeles, CA|           33602|
|         Nigeria|           32783|
|         she/her|           30341|
|           India|           29430|
| London, England|           28484|
| California, USA|           25590|
|     Chicago, IL|           25254|
|     Houston, TX|           24496|
|     Atlanta, GA|           22454|
|             USA|           21619|
|      Texas, USA|           21472|
|  Nairobi, Kenya|           19875|
|  Washington, DC|           19481|
|    Florida, USA|           19411|
|          Canada|           19400|
|  United Kingdom|           19388|
|    New York, NY|           18217|
+----------------+----------------+
only showing top 20 rows



In [17]:
df_loc.limit(10)

account_location,total_twitterers
"Stockton, CA",554
NFTCrypto is gros...,1
"Wellington City, ...",410
"St Thomas, ON",1
"Shady Side, Md.",1
Et.,1
Kirkland Washington,1
"Grover, NC",3
"Vancouver, mostly",1
kumasi,53


In [44]:
## Clean location data
na_ctry = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY','Texas','USA','US','United States',
            'Canada','canada','Toronto','toronto','Ontario','ontario',"Alaska", "Alabama",
             "Arkansas", "American Samoa", "Arizona", "California", "Colorado",
             "Connecticut", "District ", "of Columbia", "Delaware", "Florida", 
             "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana",
             "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", 
             "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina",
             "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", 
             "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", 
             "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
             "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming",
             "Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown",
             "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton",
             "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", 
             "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont",
             "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", 
             "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton",
             "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington",
             "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", 
             "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago",
             "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", 
             "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", 
             "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", 
             "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines",
             "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove",
             "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville",
             "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith",
             "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", 
             "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale",
             "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro",
             "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", 
             "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah",
             "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", 
             "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", 
             "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", 
             "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", 
             "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing",
             "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln",
             "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon",
             "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", 
             "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar",
             "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", 
             "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", 
             "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City",
             "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", 
             "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside",
             "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", 
             "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", 
             "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", 
             "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", 
             "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", 
             "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding",
             "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford",
             "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", 
             "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", 
             "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara",
             "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", 
             "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", 
             "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", 
             "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", 
             "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa",
             "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton",
             "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", 
             "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", 
             "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", 
             "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown",
          "Pacific no"]


In [45]:
# Making a lists of all countries in all continents
na_ctry1='|'.join([c for c in na_ctry])

af_ctry = my_world.get_countries_list_of('Africa')
af_ctry1='|'.join([c for c in af_ctry])

eu_ctry0 = my_world.get_countries_list_of('Europe')
eu_ctry1 = [*eu_ctry0,*['London','England','UK','United Kingdom','Scotland','Northern Ireland']]
eu_ctry2='|'.join([c for c in eu_ctry1])

oc_ctry = my_world.get_countries_list_of('Oceania')
oc_ctry1='|'.join([c for c in oc_ctry])

sa_ctry = my_world.get_countries_list_of('South America')
sa_ctry1='|'.join([c for c in sa_ctry])

as_ctry = my_world.get_countries_list_of('Asia')
as_ctry1='|'.join([c for c in as_ctry])

df_loc1 = df_loc\
.withColumn('Region',\
            when(df_loc['account_location'].rlike(af_ctry1), 'Africa')\
            .when(df_loc['account_location'].rlike(sa_ctry1), 'South America')\
            .when(df_loc['account_location'].rlike(oc_ctry1), 'Oceania')\
            .when(df_loc['account_location'].rlike(eu_ctry2), 'Europe')\
            .when(df_loc['account_location'].rlike(as_ctry1), 'Asia')\
            .when(df_loc['account_location'].rlike(na_ctry1), 'North America')\
            .otherwise(df_loc['account_location']))

In [46]:
df_loc1.limit(20)

account_location,total_twitterers,Region
"Gainesville, FL",1323,North America
...,278,...
"Alaska, USA... mo...",1,North America
Louis Tomlinson,1,Louis Tomlinson
England,353,Europe
Greater America,2,Greater America
"Brisbane, Australia",534,Oceania
"Caracas, Venezuela",482,South America
Sapno ki duniya,2,Sapno ki duniya
"Semmes, Alabama",2,North America


In [47]:
cont = ['Africa','North America','South America','Asia','Europe','Oceania']
cont1='|'.join([c for c in cont])
cont1

'Africa|North America|South America|Asia|Europe|Oceania'

In [48]:
df_loc1_conts=df_loc1.filter(df_loc1['Region'].rlike(cont1))

In [49]:
conts_acc = df_loc1_conts.groupBy('Region').agg(sum('total_twitterers').alias('total_twitterers'))\
.orderBy('total_twitterers',ascending=False).toPandas()

In [51]:
conts_acc.head(6)

,Region,total_twitterers
0,North America,1936073
1,Europe,413332
2,Asia,330740
3,Africa,320432
4,Oceania,38538
5,South America,28898


In [ ]:
sns.set_style("white")

_, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 7))
#sns.barplot(x='tweet_type',data=num_tweets, ax=axes, palette='pastel',order=num_tweets['count'].value_counts().index)

sns.barplot(x="Region", y="total_twitterers", data=conts_acc.head(6), palette="pastel")

#axes.set_xticklabels(x_labels)

plt.title('Continents - Most number of Twitterers')
plt.xlabel('Continents')
plt.ylabel('Number of Twitterers')

sns.despine()
plt.xticks(rotation=45)

plt.show()

### Country-wise analysis

In [16]:
pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 49.3 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=d98f048f098ba215c8d479685e26302ed121d3bc8e61f28641c70b0fb1a10cec
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pycountry

In [53]:
all_ctry = [country.name for country in pycountry.countries]
all_ctry = '|'.join([(country) for country in all_countries])

In [14]:
na_ctry = my_world.get_countries_list_of('North America')
sa_ctry = my_world.get_countries_list_of('South America')
eu_ctry = my_world.get_countries_list_of('Europe')
as_ctry = my_world.get_countries_list_of('Asia')
oc_ctry = my_world.get_countries_list_of('Oceania')
af_ctry = my_world.get_countries_list_of('Africa')

# Combine all country lists into a single list
all_ctry = na_ctry + sa_ctry + eu_ctry + as_ctry + oc_ctry + af_ctry

In [61]:
import pyspark.sql.functions as F
get_country = F.udf(lambda loc: next((c for c in all_ctry if loc and c.lower() in loc.lower()), None))

# Add a new column that contains the name of the matching country
df_loc1 = df_loc.withColumn('country', get_country('account_location'))

# Show the first 10 rows of the DataFrame
df_loc1.limit(10)


account_location,total_twitterers,country
"Québec, Canada",593,Canada
‘Politics Means P...,1,null
"Bengaluru, India",5239,India
"Chandigarh, India",997,India
"The Cenotaph, Hon...",1,Hong Kong
"Loveland, CO",143,null
what's the point ...,1,null
Somewhere in the ...,213,null
"Cleveland, OH",4583,null
"Rio de Janeiro, B...",1506,null


In [67]:
usa = [ 'USA','US','United States','United States of America','AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', "Aberdeen", "Abilene", "Akron", "Albany", 
             "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", 
             "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", 
             "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", 
             "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", 
             "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", 
             "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", 
             "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", 
             "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", 
             "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", 
             "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", 
             "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", 
             "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown",
       "Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

#get_usa = F.udf(lambda loc: next((c for c in usa if loc), None))
#df_loc2 = df_loc1.withColumn('usa', get_usa('account_location'))

usa_st='|'.join([c for c in usa])


df_loc2=df_loc1\
.withColumn('usa',\
            when(df_loc1['account_location'].rlike(usa_st), 'United States')\
            .otherwise(None))

df_loc2.limit(20)


account_location,total_twitterers,country,usa
Third Rock from t...,31,null,null
"Bengaluru, India",5239,India,null
South Carolina,156,null,United States
"Cleveland, OH",4583,null,United States
Fremont OH,1,null,United States
𝚀𝚒𝚗,1,null,null
"Cape Coast, Ghana",221,Ghana,null
recorriendo el mu...,1,null,null
"Patna, India",1537,India,null
Atlantis,41,null,null


In [73]:
df_loc3 = df_loc2\
.withColumn('country_final',\
            when(df_loc2['country']!='null',df_loc2['country'])\
            .when(df_loc2['usa']!='null',df_loc2['usa'])\
            .otherwise(None))

In [74]:
df_loc3.limit(10)

account_location,total_twitterers,country,usa,country_final
Grandma's SUV,1,null,null,null
"Brooklyn, NYC ➡️ ...",1,null,United States,United States
Newcastle Upon Ty...,1353,null,null,null
Trinidad,163,null,null,null
"Reno, NV",1122,null,United States,United States
"Doylestown, Pa",2,null,null,null
TRUMP LAND 🚂💨💨...,1,null,United States,United States
Getting 1% better...,1,null,null,null
南京玄武,1,null,null,null
"Natick, MA",64,null,United States,United States


In [75]:
cont_acc=df_loc3.groupBy('country_final').agg(sum('total_twitterers').alias('total_twitterers'))\
.orderBy('total_twitterers',ascending=False).toPandas()

In [77]:
cont_acc.head(20)

,country_final,total_twitterers
0,None,4965440
1,United States,1884176
2,India,175063
3,Niger,139407
4,Canada,57861
5,South Africa,55738
6,United Kingdom,46036
7,Kenya,45846
8,Pakistan,40208
9,Australia,30430


In [78]:
cont_acc.to_csv('countries_users.csv', index=False)


### Adding location to the main data frame

In [13]:
twitter_df5_loc=twitter_df5.select('id','tweet_type','account_name','account_location')
twitter_df5_loc.limit(10)

id,tweet_type,account_name,account_location
1539335991672000512,reply,GigadanielO,null
1539335994729811970,retweet,CanadianHonours,"Ottawa, Ontario"
1539335994503311361,retweet,_iwannaRAVE,Columbia.
1539335994981330944,original_tweet,AndyVermaut,"Diksmuide, België"
1539335995199479809,quote,sarge_ye,null
1539335995845402624,retweet,NEAToday,Washington DC
1539335997506428928,retweet,EvanMHarris,NJ
1539335997510623233,reply,TexasLucha,"Austin, TX"
1539335997531557888,reply,AyeTok02463593,"Ankara, Türkiye"
1539335999616126979,retweet,funmartman,null


In [15]:
import pyspark.sql.functions as F
get_country = F.udf(lambda loc: next((c for c in all_ctry if loc and c.lower() in loc.lower()), None))

# Add a new column that contains the name of the matching country
twitter_df5_loc = twitter_df5_loc.withColumn('country', get_country('account_location'))

# Show the first 10 rows of the DataFrame
twitter_df5_loc.limit(10)

id,tweet_type,account_name,account_location,country
1539335991672000512,reply,GigadanielO,null,null
1539335994729811970,retweet,CanadianHonours,"Ottawa, Ontario",null
1539335994503311361,retweet,_iwannaRAVE,Columbia.,null
1539335994981330944,original_tweet,AndyVermaut,"Diksmuide, België",null
1539335995199479809,quote,sarge_ye,null,null
1539335995845402624,retweet,NEAToday,Washington DC,null
1539335997506428928,retweet,EvanMHarris,NJ,null
1539335997510623233,reply,TexasLucha,"Austin, TX",null
1539335997531557888,reply,AyeTok02463593,"Ankara, Türkiye",null
1539335999616126979,retweet,funmartman,null,null


In [16]:
usa = [ 'USA','US','United States','United States of America','AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', "Aberdeen", "Abilene", "Akron", "Albany", 
             "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", 
             "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", 
             "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", 
             "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", 
             "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", 
             "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", 
             "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", 
             "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", 
             "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", 
             "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", 
             "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", 
             "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown",
       "Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

usa_st='|'.join([c for c in usa])


twitter_df5_loc=twitter_df5_loc\
.withColumn('usa',\
            when(twitter_df5_loc['account_location'].rlike(usa_st), 'United States')\
            .otherwise(None))

twitter_df5_loc.limit(10)

id,tweet_type,account_name,account_location,country,usa
1539335991672000512,reply,GigadanielO,null,null,null
1539335994729811970,retweet,CanadianHonours,"Ottawa, Ontario",null,United States
1539335994503311361,retweet,_iwannaRAVE,Columbia.,null,United States
1539335994981330944,original_tweet,AndyVermaut,"Diksmuide, België",null,null
1539335995199479809,quote,sarge_ye,null,null,null
1539335995845402624,retweet,NEAToday,Washington DC,null,United States
1539335997506428928,retweet,EvanMHarris,NJ,null,United States
1539335997510623233,reply,TexasLucha,"Austin, TX",null,United States
1539335997531557888,reply,AyeTok02463593,"Ankara, Türkiye",null,null
1539335999616126979,retweet,funmartman,null,null,null


In [17]:
twitter_df5_loc = twitter_df5_loc\
.withColumn('country_final',\
            when(twitter_df5_loc['country']!='null',twitter_df5_loc['country'])\
            .when(twitter_df5_loc['usa']!='null',twitter_df5_loc['usa'])\
            .otherwise(None))

twitter_df5_loc.limit(10)

id,tweet_type,account_name,account_location,country,usa,country_final
1566378682318462976,retweet,erin_gilbey,"Birmingham, England",null,United States,United States
1566378688698097664,retweet,Odeshiye,Ghana,Ghana,null,Ghana
1566378715709427719,retweet,EuroAmerican87,Anywhere but here...,null,United States,United States
1566378716887916546,retweet,Bharath12450842,null,null,null,null
1566378724135555072,retweet,lvchdvmp,archen's heart,null,null,null
1566378734529101824,retweet,doyiea_,null,null,null,null
1566378741508734976,reply,thxteacher,null,null,null,null
1566378743710744576,retweet,shoaibAlam28,"Jamui, India",India,null,India
1566378745539469312,quote,PaperupPh,10AM-10PM,null,null,null
1566378750090027008,retweet,ResisterNell,Kentucky,null,United States,United States


In [20]:
twitter_df5_loc_orig=twitter_df5_loc.filter(col('tweet_type')=='original_tweet')
twitter_df5_loc_orig.limit(10)

id,tweet_type,account_name,account_location,country,usa,country_final
1566378788845506560,original_tweet,BabichZarzycka,"Wrocław, Polska",null,null,null
1566378804402323458,original_tweet,aqeeltweet,"Gulbarga, India",India,null,India
1566378865068498947,original_tweet,fahmiswag,المملكة العربية ا...,null,null,null
1566379116017901568,original_tweet,EssayPaperUK,"Long Beach, CA",null,United States,United States
1566379138956423169,original_tweet,usualsy,6 ongoings,null,null,null
1566379279440650240,original_tweet,OlumoRocktv,Ireland,Ireland,null,Ireland
1566379420201410560,original_tweet,getthatrightgtr,Global,null,null,null
1566379426945835009,original_tweet,suppiluliumass,null,null,null,null
1566379643413880837,original_tweet,ShaikhaBinjasim,Kuwait,Kuwait,null,Kuwait
1566379715438321666,original_tweet,gotsvnteen,exo got7 svt ts,null,null,null


In [22]:
twitter_df5_loc_orig_usa=twitter_df5_loc_orig.filter(col('country_final')=='United States')
tweeters_usa=twitter_df5_loc_orig_usa.select('account_name','country_final').groupBy('account_name','country_final').count().orderBy('count',ascending=False).limit(10).toPandas()


In [23]:
tweeters_usa.head(10)

,account_name,country_final,count
0,group_kq,United States,4092
1,DennisStemmle,United States,2676
2,dealsoftakis,United States,2475
3,Rdene915,United States,1554
4,abook_and_bev,United States,1384
5,usasharenews,United States,1335
6,KandusDakus,United States,1322
7,InHomeTutor808,United States,1127
8,Bgm117771,United States,990
9,sux4_ever,United States,932


In [24]:
twitter_df5_loc_orig_ind=twitter_df5_loc_orig.filter(col('country_final')=='India')
tweeters_ind=twitter_df5_loc_orig_ind.select('account_name','country_final').groupBy('account_name','country_final').count().orderBy('count',ascending=False).limit(10).toPandas()
tweeters_ind.head(10)

,account_name,country_final,count
0,angieshandmade1,India,555
1,malpani,India,411
2,PophaleSamarth,India,378
3,school_finds,India,367
4,sirimahanthesh,India,365
5,shubhambagde00,India,351
6,htTweets,India,338
7,Granthshalaind,India,299
8,indcareer,India,285
9,sameepshastri,India,243


In [25]:
twitter_df5_loc_orig_nig=twitter_df5_loc_orig.filter(col('country_final')=='Niger')
tweeters_nig=twitter_df5_loc_orig_nig.select('account_name','country_final').groupBy('account_name','country_final').count().orderBy('count',ascending=False).limit(10).toPandas()


,account_name,country_final,count
0,angieshandmade1,India,555
1,malpani,India,411
2,PophaleSamarth,India,378
3,school_finds,India,367
4,sirimahanthesh,India,365
5,shubhambagde00,India,351
6,htTweets,India,338
7,Granthshalaind,India,299
8,indcareer,India,285
9,sameepshastri,India,243


In [26]:
tweeters_nig.head(10)

,account_name,country_final,count
0,studyinnaija,Niger,5149
1,GraceSmartsBlog,Niger,575
2,StrawberryNG,Niger,403
3,myschoolnewstv,Niger,331
4,realminablog,Niger,331
5,PelMedic,Niger,254
6,ThomasEAbiodun,Niger,229
7,MyJobMag,Niger,177
8,9jajobinfo,Niger,172
9,updatefly,Niger,171
